 # Opis notatnika
 Analiza przygotowana w poprzednim kroku została odebrana pozytywnie. W związku z tym, zostaliśmy poproszeni
 o przygotowanie raportu na podstawie danych z roku 2020. Ma być on później  wdrożony przez zespół IT na zewnętrzny
 serwis. Wybór padł na `Dash`.

 Zadanie wykonamy w dwóch krokach:
 1. Znając wymagania raportu, stworzymy na bazie danych odpowiednie komponenty, które zostaną następnie wykorzystane do wizualizacji.
 1. Stworzymy raport używając `Dash`.

 Ten notatnik skupia się na odpowiednim przygotowaniu bazy danych pod cele raportowe.

 # Wymagania biznesowe
 Po prezentacji wyników analizy na niezależnym spotkaniu, zostały określone następujące obszary raportowania:
 1. Wyświetlanie TOP 10 lotnisk na podstawie liczby odlotów, wyświetlona ma być również informacja o liczbie przylotów - widok `top_airports_by_departure`.
 1. Wyświetlenie TOP 10 tras na podstawie niezawodności, minimalna liczba lotów odbytych na trasie powinna wynosić co najmniej 10000. Przy czym istotna dla nas jest kolejność, przykładowo trasa (Warszawa, Paryż) jest inna niż (Paryż, Warszawa) - widok `top_reliability_roads`.
 1. Porównanie roku 2019 vs. 2020, aby śledzić wpływ COVID na realizację lotów. Interesują nas podejścia:
     - miesiąc do miesiąca, przykładowo styczeń 2019 vs. styczeń 2020, luty 2019 vs. luty 2020 itd. - widok `year_to_year_comparision`,
     - dzień do dnia, przykładowo wtorek 2019 vs. wtorek 2020 - widok `day_to_day_comparision`.
 1. Dzienny.

 # Podejście techniczne do problemu
 Naszym celem będzie odseparowanie warstwy przygotowania danych (logika raportu) od warstwy prezentacyjnej (wizualizacja).
 Chcemy zapewnić, aby odpowiednie procesy zajmowały się tylko swoimi zadaniami. W ten sposób warstwa prezentacyjna powinna w jak najmniejszym stopniu przetwarzać otrzymane dane.

 To podejście będzie spójne ze współczesnym sposobem projektowania aplikacji. Ma to też dodatkowy benefit - ze względu na wolumen danych na bazie, nie musimy ich najpierw pobierać - warstwa logiczna je odpowiednio zagreguje i przekaże zdecydowanie mniejszą liczbę wierszy, co przyśpieszy działanie całości.

 > Logika działania tego notebooka jest zbieżna z tą, którą robiliśmy już na przykładzie `Inicjowania bazy danych` - warto mieć go pod ręką.

 # Przygotowanie bazy danych
 Na bazie danych, gdzie umieszczone są już dane, wszystko wylądowało na schemacie `public`. Ponieważ mamy już dedykowany obszar wykorzystania danych, stworzymy sobie schemat dedykowany - `reporting`.
 Dalej stworzymy widoki, które odpowiedzą na zadane wcześniej pytania

 ## Stworzenie dedykowanego schematu
 W pliku `reporting.sql` napisz kwerendę, która stworzy (o ile już nie istnieje) schemat `reporting`.
 Ten temat nie był omawiany w trakcie trwania kursu, jednak łatwo można uzupełnić wiedzę czytając np. [ten](https://www.postgresqltutorial.com/postgresql-administration/postgresql-create-schema/) samouczek.

 # Aktualizacja bazy danych
 W tym miejscu odpowiednio skonfiguruj połączenie do bazy danych.

 Tutaj zaimportuj potrzebne biblioteki

In [1]:
import sys
sys.path.append('../')

from sqlalchemy import create_engine

from config.DB_connect import username, passwd, hostname, db_name, port

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [2]:
url = f"postgresql://{username}:{passwd}@{hostname}:{port}/{db_name}"
engine = create_engine(url, echo=False)

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [3]:
con = engine.connect()
cursor = con.connection.cursor()

 ## Wczytanie pliku `reporting.sql`
 Z katalogu `sql` wczytaj plik `reporting.sql`

In [4]:
try:
    with open("../sql/reporting.sql", 'r') as file:
        sqls = file.read()
except FileNotFoundError:
    print("File not found. Please check the file path.")
except IOError:
    print("Error occurred while reading the file.")

 W tym miejscu odpowiednio rozdziel zawartość pliku `reporting.sql` na mniejsze kwerendy używając `;`

In [5]:
sql_list = sqls.split(";")



In [11]:
sql_list

['-- CONNECTION: name=localhost\n/*\nTutaj zdefiniuj schemę `reporting`\n*/\nCREATE SCHEMA IF NOT EXISTS "reporting"\n',
 '\n/*\nTutaj napisz definicję widoku reporting.flight, która:\n- będzie usuwać dane o lotach anulowanych `cancelled = 0`\n- będzie zawierać kolumnę `is_delayed`, zgodnie z wcześniejszą definicją tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)\n\nWskazówka:\n- SQL - analiza danych > Dzień 4 Proceduralny SQL > Wyrażenia warunkowe\n- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki\n*/\nCREATE OR REPLACE VIEW reporting.flight as\nSELECT\n\tid\n\t, to_date(concat("year", LPAD("month"::TEXT, 2, \'0\'), LPAD("day_of_month"::TEXT, 2, \'0\'))\t, \'YYYYMMDD\') AS flight_date\n\t,day_of_week\n\t,op_unique_carrier\n\t,tail_num\n\t,op_carrier_fl_num\n\t,origin_airport_id\n\t,dest_airport_id\n\t,dep_time\n\t,dep_delay_new\n\t,dep_time_blk\n\t,crs_arr_time\n\t,crs_elapsed_time\n\t,crs_dep_time\n\t,arr_time\n\t,arr_delay_new\n\t,arr_time_blk\n\t,

 W tym miejscu wykonaj każdą z kwerend, aby zainicjować strukturę bazy danych

In [12]:
for sql in sql_list:
    # cursor.execute(sql)
    print(sql)

-- CONNECTION: name=localhost
/*
Tutaj zdefiniuj schemę `reporting`
*/
CREATE SCHEMA IF NOT EXISTS "reporting"


/*
Tutaj napisz definicję widoku reporting.flight, która:
- będzie usuwać dane o lotach anulowanych `cancelled = 0`
- będzie zawierać kolumnę `is_delayed`, zgodnie z wcześniejszą definicją tj. `is_delayed = 1 if dep_delay_new > 0 else 0` (zaimplementowana w SQL)

Wskazówka:
- SQL - analiza danych > Dzień 4 Proceduralny SQL > Wyrażenia warunkowe
- SQL - analiza danych > Przygotowanie do zjazdu 2 > Widoki
*/
CREATE OR REPLACE VIEW reporting.flight as
SELECT
	id
	, to_date(concat("year", LPAD("month"::TEXT, 2, '0'), LPAD("day_of_month"::TEXT, 2, '0'))	, 'YYYYMMDD') AS flight_date
	,day_of_week
	,op_unique_carrier
	,tail_num
	,op_carrier_fl_num
	,origin_airport_id
	,dest_airport_id
	,dep_time
	,dep_delay_new
	,dep_time_blk
	,crs_arr_time
	,crs_elapsed_time
	,crs_dep_time
	,arr_time
	,arr_delay_new
	,arr_time_blk
	,cancelled
	,actual_elapsed_time
	,distance
	,distance_group
	, CA

 Zatwierdzenie wszystkich operacji na bazie, czyli stworzenie widoków

In [ ]:
con.commit()

 ### Sprawdzenie

In [ ]:
# Ten kod chyba wygląda znajomo....
# istnienie widoków możemy sprawdzić tak samo jak tabele
def check_if_table_exists(table_name):
    msg = f"Sprawdzam, czy istnieje tabela {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

In [ ]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [ ]:
for view in views_to_test:
    check_if_table_exists(view)

In [ ]:
con.close()
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego zadania."
print(msg)

 # Podsumowanie
 W tym notatniku stworzyliśmy nowy schemat - `reporting`, którego zadaniem jest przygotowanie naszych danych
 do wizualizacji. Dalsza część pracy będzie polegała na wyświetleniu w wizualnie atrakcyjny sposób danych w interaktywnym raporcie stworzonym
 za pomocą `Dash`.